## Background:
This code is a implementation of the Computationally Symmetric Cross Validation Algorithm. This application of cross validation for market testing was inspired from the book "Testing and Tuning Market Trading Systems" by Timothy Masters, originally inspired by “The Probability of Backtest Overfitting” by David H. Bailey. 

## Code:

### Installing neccessary libraries
We will start by importing some important libraries for this implementation. 

In [802]:
import numpy as np
import math
import itertools
import yfinance as yf
import pandas as pd
import random
import warnings
import statistics

warnings.filterwarnings("ignore") # Ignoring all warnings

### ADDITION TO THE ALGORITHM FROM THE BOOK: 
Following, the stock data will be downloaded. 

In [803]:
# Installing the market data
ticker = "^GSPC"
data = yf.download(ticker, start = "1990-01-01") #Installing Apple data for our training and evaluation

[*********************100%***********************]  1 of 1 completed


### Generating the parameter sets
Here the return of markets of competing trading systems will be store in a matrix. The following strategys will be evaluated: SMA cross over and RSI crossover. The granularity is going to be set to high; evaluating at every bar. Three different SMA crossover systems will be evaluated and trained with parameters. 

In [804]:
# Store previous market returns of competing trading systems in a matrix
parameter_matrix = []
max_lookback = 50 # The upper max_lookback of how large the SMA can be of
n_parameters = (max_lookback - 1) * (max_lookback - 2) / 2 # The amount of randomized parameter combinations for the random search, geometric sum. 
#random.seed(42) #setting a seed just for clarity

# A wise solution would be to exclude doubles and make sure that there are no similar in the parameter grid
while len(parameter_matrix) < n_parameters:
    MA_1 = random.randint(2, max_lookback)
    MA_2 = random.randint(2, max_lookback)

    # Making sure that they are not the same
    while MA_2 == MA_1:
        MA_2 = random.randint(2, max_lookback)

    # Sorting them in increasing order
    if MA_1 < MA_2:
        parameter_set = [MA_1, MA_2]
    else:
        parameter_set = [MA_2, MA_1]
    
    # Making sure that this randomized combination is not already in the set
    if parameter_set not in parameter_matrix:
        parameter_matrix.append(parameter_set)
    else:
        pass
else:
    pass


### A grid search of the parameter space to compute all moving averages
Now moving averages will be computed for each parameter in a grid search over the parameter space. 

In [805]:
# A percetage change column is added for calculating returns
data['Daily_Return'] = 1 + data['Close'].pct_change() # The factor change from the previous day

# Generate moving averages for these parameters
"""ADDITION TO THE ALGORITHM: Generate competing systems with moving averages."""
for i in range(2, max_lookback + 1):
    data[f'MA{i}_pct_chg'] = data['Daily_Return'].rolling(window=i).mean() # Simple moving average for i days


### Parameters
Firstly, some parameters will be stored for defining how the market systems will be evaluated. 

In [806]:
n_systems = 1176 # Number of rows (competitors)
n_blocks = 10 # Number of blocks into which cases will be partitioned (has to be even for the algorithm!!!)
#ncases = len(data) / n_blocks # Number of columns in the return matrix
"""Added to the algorithm; ncases above does not makes sense"""
ncases = len(data)
""""""

indices = np.zeros(n_blocks + 1, dtype=int) # Work vector n_blocks + 1 long, these are as integers to make sure that indices are refered correctly later in the algorithm 
lengths = np.zeros(n_blocks, dtype=int) # Work vector n_blocks long, these are as integers to make sure that indices are refered correctly later in the algorithm 
flags = np.zeros(n_blocks, dtype=int) # Work vector n_blocks long, indicating which blocks is being used for training/testing, these are as integers to make sure that indices are refered correctly later in the algorithm 
work = [] # Work vector ncases long
is_crits = [] # Work vector n_systems long, stores the score of the performance criterion for each trading system 
oos_crits = [] # Work vector n_systems long

### Splitting up the data by indicing the blocks
We are now starting with the algorithm after initializing all variables. 

In [807]:
istart = 0

for i in range(n_blocks): #Looping through all blocks of returns
    indices[i] = round(istart) # The Block starts here
    lengths[i] = (ncases - istart) / (n_blocks - i) # Each Block contains this many cases 
    istart += lengths[i] # Adding to start at the next block for the next index

"""The following was added to the index list to have and ending point for the for loop"""
indices[-1] = len(data)
indices

array([   0,  880, 1760, 2640, 3520, 4401, 5282, 6163, 7044, 7925, 8806])

We also initialize the amount of times that the best IS-performer underperforms in the OOS in compairson to the others. We also add a flag array(a binary array indicating which subsets that are used for testing and training).

In [808]:
nless = 0 # Counting the amount of times the best IS-performer is not excelling the median of the other
"""This seem unnecessary. It is from the algorithm but can be handled more efficiently, which is done later in the loop."""
"""

for i in range(n_blocks / 2): # Identification of the training set blocks
    flags[i] = 1

for i in range(n_blocks / 2): # Identification of the test set blocks
    flags[i] = 0
"""

'\n\nfor i in range(n_blocks / 2): # Identification of the training set blocks\n    flags[i] = 1\n\nfor i in range(n_blocks / 2): # Identification of the test set blocks\n    flags[i] = 0\n'

### ADDITION TO THE ALGORITHM FROM THE BOOK: 
Now we have to create an algorithm to generate all possible combinations of training and test sets. This is made instead of doing an iradix loop like the algorithm provided. The generation of permutations is handled way more efficient this way resulting in less computing time. 

In [809]:
# Generating permutations
elements = [1] * (n_blocks // 2) + [0] * (n_blocks // 2) # Generate a structure to follow for creating permutations

ncombo = list(set(itertools.permutations(elements))) # Generate all unique permutations
ncombo = np.array(list(set(itertools.permutations(elements)))) # Creating an array of it    
""""ADDITION TO THE ALGORITHM: Occurance of best parameter-set"""
parameter_occurance = np.zeros(int(n_parameters)) #Counting the occurance of each parameter set
""""""

''

### Defining whether to hold a stock or not
A implementation for defining where the trading system is active is neccessed. A binary system will be created and implemented to each strategy, where the integer indicates whether the stock is holded or not. 1 = buying/holding the stock, 0 = not holding/selling the stock. A loop through each parameter combination will be carried out where a signal column will be added where the short term moving average is higher than the long term. The performance criterion will be the total return. 

In [810]:
# For each parameter combination a hold signal is created for whether to hold or not and is stored in the returns_dataframe and in data.
maximal_return = 1
best_parameter_combination = []
returns_dataframe = data[["Daily_Return"]]
for parameter_combination in parameter_matrix:
    """Check whether these should be shifted or not, for singalling and buying the day after?"""
    data[f'MA{parameter_combination[0]}_MA{parameter_combination[1]}_Signal'] = (data[f'MA{parameter_combination[0]}_pct_chg'] > data[f'MA{parameter_combination[1]}_pct_chg']).astype(int).shift(1) # Shift because the buy/sell signal is not going to be immediate
    returns_dataframe[f'MA{parameter_combination[0]}_MA{parameter_combination[1]}_Signal'] = (data[f'MA{parameter_combination[0]}_pct_chg'] > data[f'MA{parameter_combination[1]}_pct_chg']).astype(int).shift(1)

    # Determining where the signal is 1, and calculating the return based on this
    returns_dataframe[f'system_return_MA{parameter_combination[0]}_MA{parameter_combination[1]}'] = np.where(returns_dataframe[f'MA{parameter_combination[0]}_MA{parameter_combination[1]}_Signal'] == 1, data['Daily_Return'], 1)


    """Addition to the algorithm. This is implemented just to see which parameter set gives the maximal return over the time period. This is just for exploritory purposes, deciding which trading system is yielding the best return over the entire time period."""
    # Calculating the total return for each system:
    return_of_system = returns_dataframe[f'system_return_MA{parameter_combination[0]}_MA{parameter_combination[1]}'].prod()
    if return_of_system > maximal_return:
        maximal_return = return_of_system
        best_parameter_combination = parameter_combination
        print(f'New maximal return for parameter set {best_parameter_combination} with a return of {(return_of_system - 1) * 100:.2f}%')


New maximal return for parameter set [36, 48] with a return of 155.52%
New maximal return for parameter set [20, 39] with a return of 326.01%
New maximal return for parameter set [13, 43] with a return of 349.82%
New maximal return for parameter set [14, 15] with a return of 1154.09%
New maximal return for parameter set [25, 26] with a return of 1258.73%


## Creating the loop for dividing the blocks and calculating the optimization criterion
Now the optimization criterion is going to be defined for selecting the best performing trading system. The optimization criterion will be mean return per bar of each trading system. This includes where we split our returns in training and test sets and calculate the total return for each system. 

The outermost loop will loop through all possible combinations of training and test sets, evaluating each system based on a specific criterion. In this specific case we will start by evaluating them by the win ratio of trades. 

In [811]:
# Initializing this solely for printing
subset_number = 0

"""
# Counting for evaluation of the best IS-performer
n = 0
"""
for combination in ncombo:
    """
    Here we are going to use the criterion to compute the IS-performance of all training system for this specific combination
    """
    flags[:] = combination # Define what the flag list is currently
    training_data = pd.DataFrame()
    test_data = pd.DataFrame()
    for subset_index in range(len(ncombo[0])):
        if flags[subset_index] == 1:
            training_data = pd.concat([training_data, returns_dataframe.iloc[indices[subset_index]:indices[subset_index + 1]]]) #.reset_index(drop=True) # Perhaps include reset_index, does not really matter since the index is not used as a parameter
        else:
            test_data = pd.concat([test_data, returns_dataframe.iloc[indices[subset_index]:indices[subset_index + 1]]]) #.reset_index(drop=True) # Perhaps include reset_index, does not really matter since the index is not used as a parameter

    """Now the subsets is divided for each combination and the training can start. Here the IS-performance of all the competing systems will be calculated."""

    """Now the evaluation of the test set will take place and the trading systems will be ranked. """

    """Implement a calculation of IS- and OOS-performance, storing a score in the is_crits and oos_crits work vector, average return per bar is supposed to be the performace criterion, sum up all the returns and divide by the lenght of the block"""
    is_crits = []
    oos_crits = []
    for parameter_combination in parameter_matrix:
        is_crits.append((training_data[f'system_return_MA{parameter_combination[0]}_MA{parameter_combination[1]}'].prod() - 1) / len(returns_dataframe) + 1) #The performance criterion is set to be the mean return per bar
        oos_crits.append((test_data[f'system_return_MA{parameter_combination[0]}_MA{parameter_combination[1]}'].prod() - 1) / len(returns_dataframe) + 1) # The Out of sample performance for each competing system

    #Finding the maximum value
    best_is_crits = max(is_crits)
    max_index = is_crits.index(best_is_crits)
    
    # Calculating the median of the OOS-performances
    median_oos_crits = statistics.median(oos_crits)
    closest_value = min(oos_crits, key=lambda x: abs(x - median_oos_crits)) # Finding the closest value due error that occured when working with float numbers
    median_index = oos_crits.index(closest_value)

    # Determines whether the best IS-performer's OOS-performance exceeds the median and counts the amount of times it does
    if oos_crits[max_index] <= oos_crits[median_index]:
        nless += 1
    
    """# Computes the relative rank deciding if the best IS-performer is ranked compared to the other competitors 
    rel_rank = n / (n_systems + 1)

    if rel_rank <= 0.5: # Is the relative rank below the median?
        nless += 1
        """

    # Counter for printing purposes
    subset_number += 1

    # Highest occuring parameter combination
    parameter_occurance[max_index] += 1

    print(f'Subset {subset_number}/{len(ncombo)}: The best IS-performance is {best_is_crits}, with the parameter set {parameter_matrix[max_index]}, yielding an OOS-performance of {oos_crits[max_index]}, with the median OOS-performance {median_oos_crits}. \nThe training/test split was: {flags}.\n')

Subset 1/252: The best IS-performance is 1.0005888088465889, with the parameter set [33, 34], yielding an OOS-performance of 0.9999964947707877, with the median OOS-performance 1.0000557876944538. 
The training/test split was: [0 1 1 0 0 1 1 0 0 1].

Subset 2/252: The best IS-performance is 1.0003062488356482, with the parameter set [14, 15], yielding an OOS-performance of 1.0002716710795532, with the median OOS-performance 1.0001623323225821. 
The training/test split was: [1 1 0 1 0 1 0 1 0 0].

Subset 3/252: The best IS-performance is 1.0004115834370664, with the parameter set [44, 50], yielding an OOS-performance of 1.000061224085304, with the median OOS-performance 1.0000622809782183. 
The training/test split was: [0 1 0 0 1 1 0 1 1 0].

Subset 4/252: The best IS-performance is 1.0009438477259023, with the parameter set [24, 25], yielding an OOS-performance of 0.9999875083175096, with the median OOS-performance 1.0000085280452222. 
The training/test split was: [1 1 1 0 0 0 1 0 0 1]

## Calculating the probability

In [812]:


# Calculating the probability of underperformance
value = nless / len(ncombo)

# Most frequent occuring parameter set
max_parameter_occurance = max(parameter_occurance)
max_parameter_occurance_index = np.argmax(parameter_occurance)

print(f'The estimated probability of underperformance for this strategy is {value * 100:.2f}%, for the {ticker} market, with verification on {len(ncombo)} subset combinations.')
print(f'Some parameters for the experiment is: \nNumber of blocks for pooling: {n_blocks} \nAmount of competing systems: {int(n_parameters)} \nLength of dataset: {len(data)} \nMost frequent occuring parameter set: {parameter_matrix[max_parameter_occurance_index]} which occured {int(max_parameter_occurance)}/{len(ncombo)} times')

The estimated probability of underperformance for this strategy is 33.33%, for the ^GSPC market, with verification on 252 subset combinations.
Some parameters for the experiment is: 
Number of blocks for pooling: 10 
Amount of competing systems: 1176 
Length of dataset: 8806 
Most frequent occuring parameter set: [14, 15] which occured 57/252 times
